<a href="https://colab.research.google.com/github/tramyynt/Stress-Analysis/blob/main/Stress_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using Text Mining, NLP and Classification algorithms to classify a comment on Reddit is stressed or non-stressed.



## **Data Preparation**

In [65]:
#import lib
! pip install -q kaggle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from scipy import spatial

import nltk
from nltk.corpus import stopwords
from nltk.corpus import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer



In [4]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tramyynt26","key":"d3e9f3fd41fa1c8ba08273c29eca6c96"}'}

In [5]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              233KB  2021-08-19 07:30:16          10892  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           6667  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2517  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2677  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [7]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 2.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=7feb0b6c4834fcdd237755c49a02b00c838175d07a4a4c0e34d03c883ec21bc6
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [8]:
!kaggle datasets download ruchi798/stress-analysis-in-social-media -p /content/gdrive/kaggle

  0% 0.00/1.31M [00:00<?, ?B/s]
100% 1.31M/1.31M [00:00<00:00, 123MB/s]


In [9]:
import zipfile
from google.colab import drive

#drive.mount('/content/drive/')

!unzip /content/gdrive/kaggle/stress-analysis-in-social-media.zip -d '/content/gdrive/kaggle'

Archive:  /content/gdrive/kaggle/stress-analysis-in-social-media.zip
  inflating: /content/gdrive/kaggle/dreaddit-test.csv  
  inflating: /content/gdrive/kaggle/dreaddit-train.csv  


In [10]:
train = pd.read_csv('/content/gdrive/kaggle/dreaddit-train.csv')
test = pd.read_csv('/content/gdrive/kaggle/dreaddit-test.csv')

In [ ]:
train.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,lex_liwc_WPS,lex_liwc_Sixltr,lex_liwc_Dic,lex_liwc_function,lex_liwc_pronoun,lex_liwc_ppron,lex_liwc_i,lex_liwc_we,lex_liwc_you,lex_liwc_shehe,lex_liwc_they,lex_liwc_ipron,lex_liwc_article,lex_liwc_prep,lex_liwc_auxverb,lex_liwc_adverb,lex_liwc_conj,lex_liwc_negate,lex_liwc_verb,lex_liwc_adj,lex_liwc_compare,lex_liwc_interrog,lex_liwc_number,lex_liwc_quant,lex_liwc_affect,...,lex_liwc_motion,lex_liwc_space,lex_liwc_time,lex_liwc_work,lex_liwc_leisure,lex_liwc_home,lex_liwc_money,lex_liwc_relig,lex_liwc_death,lex_liwc_informal,lex_liwc_swear,lex_liwc_netspeak,lex_liwc_assent,lex_liwc_nonflu,lex_liwc_filler,lex_liwc_AllPunc,lex_liwc_Period,lex_liwc_Comma,lex_liwc_Colon,lex_liwc_SemiC,lex_liwc_QMark,lex_liwc_Exclam,lex_liwc_Dash,lex_liwc_Quote,lex_liwc_Apostro,lex_liwc_Parenth,lex_liwc_OtherP,lex_dal_max_pleasantness,lex_dal_max_activation,lex_dal_max_imagery,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,116,72.64,15.04,89.26,1.00,29.00,12.93,87.07,56.03,16.38,12.07,9.48,0.00,0.86,1.72,0.0,4.31,3.45,19.83,7.76,5.17,4.31,1.72,16.38,6.03,3.45,0.86,1.72,1.72,8.62,...,0.86,10.34,6.03,0.86,0.00,0.00,0.00,2.59,0.0,0.86,0.86,0.00,0.00,0.0,0.0,21.55,9.48,3.45,0.86,0.86,0.0,0.0,0.0,5.17,1.72,0.00,0.00,2.8571,2.6250,3.0,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,109,79.08,76.85,56.75,98.18,27.25,21.10,87.16,48.62,11.93,7.34,1.83,2.75,2.75,0.00,0.0,4.59,8.26,13.76,6.42,3.67,8.26,0.92,15.60,2.75,0.92,0.92,2.75,0.92,5.50,...,2.75,10.09,1.83,11.01,0.00,0.00,0.92,0.00,0.0,1.83,0.00,0.92,0.00,0.0,0.0,14.68,4.59,2.75,0.00,0.00,0.0,0.0,0.0,0.00,2.75,0.92,3.67,3.0000,2.8889,3.0,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,167,33.80,76.38,86.24,25.77,33.40,17.37,91.02,61.68,25.15,16.17,8.98,1.80,1.80,2.99,0.6,8.98,5.39,12.57,10.18,1.80,5.99,1.20,20.96,1.20,0.60,0.60,1.20,1.80,2.40,...,4.79,5.99,5.39,0.00,1.20,0.60,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,10.78,2.40,3.59,0.00,0.00,0.6,0.0,0.0,2.40,1.80,0.00,0.00,2.7143,3.0000,3.0,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,273,2.98,15.25,95.42,79.26,54.60,8.06,98.90,65.57,30.40,23.44,16.12,0.37,0.37,6.59,0.0,6.96,3.30,9.16,8.79,6.59,9.89,3.66,20.88,3.66,1.83,1.10,0.00,1.10,8.79,...,1.83,3.30,6.23,0.73,0.37,0.37,0.00,0.00,0.0,2.56,0.00,0.73,0.73,0.0,0.0,12.09,2.56,7.33,0.00,0.00,0.0,0.0,0.0,1.47,0.00,0.73,0.00,3.0000,2.6364,3.0,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,89,32.22,28.71,84.01,1.00,17.80,31.46,88.76,52.81,15.73,11.24,7.87,0.00,0.00,3.37,0.0,4.49,4.49,8.99,13.48,4.49,4.49,2.25,13.48,4.49,2.25,1.12,1.12,1.12,7.87,...,2.25,2.25,10.11,1.12,0.00,2.25,1.12,0.00,0.0,1.12,0.00,0.00,0.00,0.0,0.0,16.85,5.62,6.74,1.12,0.00,0.0,0.0,0.0,0.00,1.12,2.25,0.00,3.0000,3.0000,3.0,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [ ]:
subred = df['subreddit']

In [ ]:
len(train['text'])

2838

In [ ]:
subred.unique()

array(['ptsd', 'assistance', 'relationships', 'survivorsofabuse',
       'domesticviolence', 'anxiety', 'homeless', 'stress',
       'almosthomeless', 'food_pantry'], dtype=object)

## **Data Preprocessing** 

Data preprocessing before going through vectorizing words.

1.   Tokenization, Removal of punctuations.
2.   Filtering stopwords
3.   Normalization ( Stemming and Lemmatization )









### **Tokenization, Removal of punctuations**

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
tokenizer = RegexpTokenizer(r'[a-zA-Z]{2,}') #remove number and words has only one character
# sr = []
# for x in range(0,len(train['text'])):
#   dt = tokenizer.tokenize(train['text'][x])
#   sr.append(dt)
train['processed_text'] = train['text'].apply(lambda x: tokenizer.tokenize(x.lower()))
test['processed_text'] = test['text'].apply(lambda x: tokenizer.tokenize(x.lower()))

### **Filtering stopwords**

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
def filtering_stopwords(data):
  words = [w for w in data if not w in stopwords.words('english')]
  return words

In [15]:
train['processed_text'] = train['processed_text'].apply(lambda x: filtering_stopwords(x))
test['processed_text'] = test['processed_text'].apply(lambda x: filtering_stopwords(x))


### **Normalization**

In [16]:
stemmer = PorterStemmer()
def word_stemmer(data):
    stem_text = [stemmer.stem(i) for i in data]
    return stem_text

train['processed_text'] = train['processed_text'].apply(lambda x: word_stemmer(x))
test['processed_text'] = test['processed_text'].apply(lambda x: word_stemmer(x))

In [ ]:
train['processed_text']

0       [said, felt, way, sugget, go, rest, trigger, a...
1       [hey, assist, sure, right, place, post, goe, c...
2       [mom, hit, newspap, shock, would, know, like, ...
3       [met, new, boyfriend, amaz, kind, sweet, good,...
4       [octob, domest, violenc, awar, month, domest, ...
                              ...                        
2833    [week, ago, preciou, ignor, jan, happi, year, ...
2834    [abil, cope, anymor, tri, lot, thing, trigger,...
2835    [case, first, time, read, post, look, peopl, w...
2836    [find, normal, good, relationship, main, probl...
2837    [talk, mom, morn, said, sister, trauma, wors, ...
Name: processed_text, Length: 2838, dtype: object

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [18]:
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(data):
  lemma_text = [lemmatizer.lemmatize(i) for i in data]
  return lemma_text
train['processed_text'] = train['processed_text'].apply(lambda x: word_lemmatizer(x))
test['processed_text'] = test['processed_text'].apply(lambda x: word_lemmatizer(x))

In [ ]:
train['processed_text']

0       [said, felt, way, sugget, go, rest, trigger, a...
1       [hey, assist, sure, right, place, post, goe, c...
2       [mom, hit, newspap, shock, would, know, like, ...
3       [met, new, boyfriend, amaz, kind, sweet, good,...
4       [octob, domest, violenc, awar, month, domest, ...
                              ...                        
2833    [week, ago, preciou, ignor, jan, happi, year, ...
2834    [abil, cope, anymor, tri, lot, thing, trigger,...
2835    [case, first, time, read, post, look, peopl, w...
2836    [find, normal, good, relationship, main, probl...
2837    [talk, mom, morn, said, sister, trauma, wors, ...
Name: processed_text, Length: 2838, dtype: object

## **Word Embedding**

Using Global Vectors for Word Representation ( GLOVE) is an “unsupervised learning algorithm for obtaining vector representations for words "
Reference:
https://towardsdatascience.com/understanding-word-embeddings-with-tf-idf-and-glove-8acb63892032
https://edumunozsala.github.io/BlogEms/jupyter/nlp/classification/embeddings/python/2020/08/15/Intro_NLP_WordEmbeddings_Classification.html
https://gist.github.com/TomLin/30244bcccb7e4f94d191a878a697f698
https://docs.google.com/presentation/d/1Gg0l1GySbVOJ5Uys8gqfGnuHd-zIv6MZR5-ZoqxJxL4/edit#slide=id.g6e630d8087_0_181
https://edumunozsala.github.io/BlogEms/jupyter/nlp/classification/embeddings/python/2020/08/15/Intro_NLP_WordEmbeddings_Classification.html

In [29]:
from gensim.scripts.glove2word2vec import glove2word2vec

In [32]:
word2vec_output_file = 'glove.6B.100d.txt'+'.word2vec'
glove2word2vec('/content/glove.6B.100d.txt', word2vec_output_file)

(400000, 100)

In [75]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [ ]:
keys = ['abuse', 'anxiety', 'financial', 'ptsd', 'social', 'stress', 'love', 'friendship']
for i in keys:
  print('Vector of:', i ,model.get_vector(i))
  result = model.most_similar(positive=i, topn=10)
  print('\n10 most similar words to: ',i,result)

Vector of: abuse [ 0.6747     0.31766    0.19427   -0.23009   -0.7347     1.4631
 -0.6046    -0.098757   0.054897   0.23114   -0.074416   0.088503
 -0.26545    1.1766     0.32394    0.31836    0.073733  -0.019944
 -1.5462     0.44651   -0.26256   -0.26111   -0.32883    0.60584
 -0.66182    0.028531   0.61704   -0.34273   -0.20538    0.77077
  1.0128    -0.14495    0.0048883  0.63247   -0.26677   -0.19595
 -0.44454   -0.3079    -0.21728    0.38524   -0.83284    0.18319
 -0.083784  -0.66435   -0.13656    0.26515    0.34094   -0.27263
  0.0086811 -0.76744    0.83997   -0.44614   -0.71315    1.2288
  0.16899   -1.0177    -0.16239   -0.90029    1.3657     0.17637
  0.051633  -0.068679   0.16901   -0.95653    0.55789   -0.12813
  0.95465   -1.1387     0.61918    0.20519    0.18888    0.029914
 -0.68286    0.28326    0.36491    0.071344  -0.23957    0.32145
 -1.1288     0.39188    0.83201    0.45375    0.59285    0.50722
 -2.0063    -0.72023   -0.12334    0.24201   -0.24597   -0.65065
 -0.397

**TF-IDF Weighted Average**

In [66]:
class TfidfEmbeddingVectorizer(object):

    def __init__(self, word_model):

        self.word_model = word_model
        self.word_idf_weight = None
        self.vector_size = word_model.wv.vector_size

    def fit(self, docs):  # comply with scikit-learn transformer requirement
        """
        Fit in a list of docs, which had been preprocessed and tokenized,
        such as word bi-grammed, stop-words removed, lemmatized, part of speech filtered.
        Then build up a tfidf model to compute each word's idf as its weight.
        Noted that tf weight is already involved when constructing average word vectors, and thus omitted.
        :param: pre_processed_docs: list of docs, which are tokenized
        :return: self
        """

        text_docs = []
        for doc in docs:
            text_docs.append(" ".join(doc))

        tfidf = TfidfVectorizer() # default 1-gram 
        tfidf.fit(text_docs)  # must be list of text string

        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of
        # known idf's
        max_idf = max(tfidf.idf_)  # used as default value for defaultdict
        self.word_idf_weight = defaultdict(lambda: max_idf,
                                           [(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()])
        return self

    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector



    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.
        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """

        mean = []
        for word in sent:
            if word in self.word_model.wv.vocab:
                mean.append(self.word_model.wv.get_vector(word) * self.word_idf_weight[word])  # idf weighted

        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            logging.warning("cannot compute average owing to no vector for {}".format(sent))
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean


    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])        

In [82]:
tfidf_vec_tr = TfidfEmbeddingVectorizer(model)

tfidf_vec_tr.fit(train['processed_text'])  # fit tfidf model first
X_train = tfidf_vec_tr.transform(train['processed_text'])
y_train = train['label']

tfidf_vec_tr.fit(test['processed_text'])
X_test = tfidf_vec_tr.transform(test['processed_text'])
y_test = test['label']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [83]:
clf_dict = {'log reg': LogisticRegression(random_state=42), 
            'naive bayes': GaussianNB(), 
            'linear svc': LinearSVC(random_state=42),
            'sgd classifier': SGDClassifier(random_state=42),
            'ada boost': AdaBoostClassifier(n_estimators=100, random_state=42),
            'gradient boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
            'CART': DecisionTreeClassifier(random_state=42),
            'random forest': RandomForestClassifier(n_estimators=100, random_state=42)}
for name, clf in clf_dict.items():
    pred = clf.fit(X_train, y_train)
    y_pred = pred.predict(X_test)
    print('Accuracy of {}:'.format(name), accuracy_score(y_pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy of log reg: 0.6741258741258741
Accuracy of naive bayes: 0.6139860139860139


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy of linear svc: 0.6713286713286714
Accuracy of sgd classifier: 0.5720279720279721
Accuracy of ada boost: 0.6573426573426573
Accuracy of gradient boosting: 0.6671328671328671
Accuracy of CART: 0.5986013986013986
Accuracy of random forest: 0.6629370629370629
